# Convert Test Task Lefebvre

This code is used to convert the data of Lefebvre from: https://figshare.com/articles/dataset/Behavioral_data_and_data_extraction_code/4265408/1

The Data has been downloaded to `dat/lefebvre_exp` with the corresponding matlab and python code. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.io
import torch

## Population definition
exps = []

for exp in range(2): # exp 1 == reward and reward omission, exp 2 == reward and punishment
    exp+=1
    path = '../dat/lefebvre_exp/b_data/'
    n_subjects = 0
    ## Data Extraction
    if exp == 1:
        n_subjects = 50
        path += 'data_exp1'
    elif exp == 2:
        n_subjects = 35
        path += 'data_exp2'

    conditions = np.zeros((n_subjects, 96))
    choices = np.zeros((n_subjects, 96))
    reward = np.zeros((n_subjects, 96))

    for i in range(n_subjects):
        if exp == 1:
            data = scipy.io.loadmat(path + '/exp1_' + str(i+1))
            data = data['data']
            conditions[i,:] = data[:,2] # 1 to 4 as per condition
            choices[i,:] = data[:, 6] / 2 + 1.5 # 1 for left, 2 for right
            reward[i,:] = data[:, 7] / 2 # 0 or 0.5 euro
        elif exp == 2:
            data = scipy.io.loadmat(path + '/exp2_' + str(i+1)) # 1 to 4 as per condition
            data = data['data']
            conditions[i,:] = data[:,2] # 1 for left, 2 for right
            choices[i,:] = data[:, 4] / 2 + 1.5 # -0.5 or 0.5 euros
            reward[i,:] = data[:, 7]
    exps.append((conditions, choices, reward))

torch.save(exps, '../dat/lefebvre_exp/extracted_data.pt')


Following that hasn't been used but has been converted from matlab to python:

In [10]:
## This script compute and plot behavioural variable of interest.

def struc_to_plot_matrix(choices,whichcondition,conditions,subjects,trials,adjust):
    
    # with sesstions a = np.zeros((trials,subjects,sessions))
    a = np.zeros((subjects, trials))

    for sub in range(subjects):

        cond = conditions[sub,:]
        vari = choices[sub,:]
        vari = vari + adjust

        a[sub,:] = a[sub,:] + vari[np.where(cond == whichcondition)]

    return a


choice1 = struc_to_plot_matrix(choices,1,conditions,n_subjects,24,-1)
choice2 = struc_to_plot_matrix(choices,2,conditions,n_subjects,24,-1)
choice3 = struc_to_plot_matrix(choices,3,conditions,n_subjects,24,-1)
choice4 = struc_to_plot_matrix(choices,4,conditions,n_subjects,24,-1)

In [ ]:
"""
# %%
## Graph 
import matplotlib.pyplot as plt

def PlotFunction(DataMatrix,Colors,Yinf,Ysup,Font): 
    Ntrial,Nsub = DataMatrix.shape
    curve = nanmean(DataMatrix,2)
    sem = np.transpose(nanstd(np.transpose(DataMatrix))) / np.sqrt(Nsub)
    absci = np.arange(1,Ntrial+1)
    for n in np.arange(1,Ntrial+1).reshape(-1):
        # first bar
        bar(n,curve(n),'FaceColor',Colors(n,:),'EdgeColor',Colors(n,:),'BarWidth',0.75,'LineWidth',3)
        hold('on')
        errorbar(n,np.array([curve(n)]),np.array([sem(n)]),'Color',np.array([0,0,0]),'LineStyle','none','LineWidth',3)
    
    plt.ylim(np.array([Yinf,Ysup]))
    set(gca,'FontSize',Font,'XLim',np.array([0,Ntrial + 1]),'XTick',np.array([np.arange(1,Ntrial+1)]),'YTick',np.array([np.arange(- 1,1+0.1,0.1)]),'XTickLabel','')
    return Ntrial,Nsub

def PlotFunction2(DataMatrix = None,Chance = None,Color = None,Line = None,Alpha = None,Yinf = None,Ysup = None,Font = None,Title = None,LabelX = None,LabelY = None): 
    Ntrial,Nsub = DataMatrix.shape
    curve = nanmean(DataMatrix,2)
    sem = np.transpose(nanstd(np.transpose(DataMatrix))) / np.sqrt(Nsub)
    curveSup = (curve + sem)
    curveInf = (curve - sem)
    for n in np.arange(1,Ntrial+1).reshape(-1):
        chance[n,1] = Chance(1)
        chance[n,2] = Chance(2)
        chance[n,3] = Chance(3)
    
    plt.plot(curve + sem,'Color',Color,'LineWidth',Line)
    hold('on')
    plt.plot(curve - sem,'Color',Color,'LineWidth',Line)
    hold('on')
    plt.plot(curve,'B','Color',Color,'LineWidth',Line * 2)
    hold('on')
    fill(np.array([np.arange(1,Ntrial+1),np.transpose(flipud(np.transpose(np.array([np.arange(1,Ntrial+1)]))))]),np.array([np.transpose(curveSup),np.transpose(flipud(curveInf))]),'k','LineWidth',1,'LineStyle','none','FaceColor',Color,'FaceAlpha',Alpha)
    plt.plot(chance,'k','LineWidth',Line / 2)
    plt.axis(np.array([0,Ntrial + 1,Yinf,Ysup]))
    set(gca,'Fontsize',Font)
    plt.title(Title)
    plt.xlabel(LabelX)
    plt.ylabel(LabelY)
    box('ON')
    return curve,sem


# Color Definition
Colors[1,:] = np.array([0.5,0,0])
Colors[2,:] = np.array([0.5,0,0])
Colors[3,:] = np.array([0.5,0,0])
Colors[4,:] = np.array([0.5,0,0])
# Bar Plot

figure
subplot(2,2,1)

meandata = np.full([4,np.asarray(choice1(1,:)).size],np.nan)
meandata[1,:] = mean(choice1)
meandata[2,:] = mean(choice2)
meandata[3,:] = mean(choice3)
meandata[4,:] = mean(choice4)
PlotFunction(meandata,Colors,0,1,14)
plt.title('right')
plt.ylabel('Arbitrary units')
subplot(2,2,2)

meandata2 = np.zeros((4,np.asarray(choice1(1,:)).size))
meandata2[2,:] = mean(1 - choice2)
meandata2[3,:] = mean(choice3)
PlotFunction(meandata2,Colors,0,1,14)
plt.title('Correct')
subplot(2,4,5)
PlotFunction2((choice1),np.array([0.25,0.5,0.75]),np.array([0.5,0,0]),1,0.5,0,1,10,'25/25','Trials','P(right)')
subplot(2,4,6)
PlotFunction2((choice2),np.array([0.25,0.5,0.75]),np.array([0.5,0,0]),1,0.5,0,1,10,'75/25','Trials','P(right)')
subplot(2,4,7)
PlotFunction2((choice3),np.array([0.25,0.5,0.75]),np.array([0.5,0,0]),1,0.5,0,1,10,'25/75','Trials','P(right)')
subplot(2,4,8)
PlotFunction2((choice4),np.array([0.25,0.5,0.75]),np.array([0.5,0,0]),1,0.5,0,1,10,'75/75','Trials','P(right)')
"""